## Создание массива слов
Для того, чтобы находить и исправлять опечатки с помощью расстояния Левенштейна, сперва необходимо создать массив слов, чтобы мы могли найти правильно написанный вариант слова, в котором допустили опечатку.

Для этого мы извлёчем все слова из корпуса. Корпус это массив языковых данных, например письменных текстов, аудиозаписей и так далее. Корпуса представлены в электронном виде для решения разных задач. Например, для решения нашей задачи.

### Загрузка корпуса
Мы воспользуемся одним из немногих доступных корпусов английского языка — Brown Corpus. Для начала необходимо добавить корпус в google colab. Следуй инструкции:
1. Открываем files google colab (папка в панель слева).
2. Нажимаем левой кнопкой мыши по папке с названием "...".
3. Нажимаем "новая папка"/"new folder".
4. В качестве названия указываем Brown.
5. Кликаем левой кнопкой мыши по папке Brown.
6. Нажимаем "загрузить"/"upload".
7. Находим корневую папку проекта.
8. Открываем в корневой папке проекта папку Brown.
9. Выделяем все файлы комбинацией клавиш "ctrl + A".
10. Нажимаем "открыть"/"open".

После того, как все файлы прогрузятся (около минуты) перейдём к следующей задаче.

### Обработка корпуса
После того, как мы загрузили корпус, необходимо представить все эти текстовые файлы в удобном для нас формате. Проще всего будет воспользоваться функционалом библиотеки corpus_toolkit. Запусти код ниже, чтобы установить всё необходимое:

In [ ]:
!pip install corpus-toolkit  # Устанавливаем библиотеку в google colab

from corpus_toolkit import corpus_tools as ct  # Импортируем библиотеку
import spacy  # Импортируем вспомогательную библиотеку

Эта библиотека создана для создания так называемых словарей частотностей. Это такие пары ключ-значение, где ключ это слово, а значение количество появлений в корпусе. Мы создадим словарь частотности, а затем вытащим из него все ключи, воспользовавшись кодом из [документации](https://github.com/kristopherkyle/corpus_toolkit)

In [ ]:
brown_corp = ct.ldcorpus('Brown')  # Загружаем корпус
tok_corp = ct.tokenize(brown_corp)  # Токенизируем корпус
brown_freq = ct.frequency(tok_corp)  # Создаём словарь частотности

Давайте посмотрим действительно ли всё сработало, выведем самые частотные слова корпуса:

In [ ]:
ct.head(brown_freq, hits = 5)  # Вывод 5 самых частотных слов

Отлично, теперь постараемся разобраться с тем, что такое словарь и как извлечь из него ключи. Словарь это тип данных на python, который позволяет хранить пары ключ-значение, например:

In [ ]:
# Словарь иницилизируется с помощью фигурных скобок {}
grammy_awards_dict = {
    'Kanye West': 24,  # Внутри словаря мы пишем ключ, затем двоеточие,
                       # а затем значение
    'Jay-Z': 24,
    'Beyoncé': 32
}

# Выведем словарь, чтобы посмотреть, что у нас получилось
print(grammy_awards_dict)

Зная ключ словаря, мы можем найти значение. Для этого мы напишем ключ справа от переменной словаря:

In [ ]:
# Узнаем количество грэмми у Канье Веста
print(grammy_awards_dict['Kanye West'])

Обратись к словарю частотности `brown_freq` и узнай частотность какого-нибудь английского слова:

Также мы можем отдельно получить все ключи или значения словаря, или и те и другие:

In [ ]:
print(grammy_awards_dict.keys())  # Выведем все ключи

In [ ]:
print(grammy_awards_dict.values())  # Выведем все значения

In [ ]:
print(grammy_awards_dict.items())  # Выведем все пары ключ-значение

Отлично! Нас интересуют только ключи словаря частотности, но нам не подходит формат dict_keys (он указан в выводе ключей `grammy_awards_dict`). Сделаем из этого список и сохраним в отдельной переменной

In [ ]:
grammy_awards_keys = list(grammy_awards_dict.keys())

print(grammy_awards_keys)  # Теперь у нас есть list, то есть список.
                           # Со списком работать гораздо удобнее

Теперь сделай список ключей для словаря частотности `brown_freq`:

Мы можем узнать, как много ключей в словаре, вызвав функцию
`len()` и поместив в круглые скобки наш список ключей, напиши это ниже:

Отлично, у тебя прекрасно получается!

### Сохранение списка слов в виде json массива
Если мы каждый раз будем загружать весь корпус для того, чтобы извлечь список, у нас будет уходить на это несколько секунд. Кажется, что это немного, но кто станет ждать 5 секунд, чтобы увидеть свои опечатки? Мы воспользуемся одним из самых удобных форматов хранения данных — json.

JSON (JavaScript Object Notation) — текстовый формат обмена данными. Бывают json объекты, массивы, строки, числа и литералы. Нас будет интересовать json массив, то есть просто список элементов. В качестве элементов будут наши слова.

Но для начала разберёмся с тем, как на python работать с файлами. Всё, что нужно сейчас это простая конструкция ниже:

In [ ]:
with open('Brown/ca_ca01.txt', 'r') as file:
  print(file.read())

Мы открыли файл из корпуса, указав в аргументе функции open() путь к файлу в кавычках и способ взаимодействия: `'r'` это чтение (read), `'w'` это чтение и ввод (write). Если мы укажем `'w'` вместо `'r'`, то при отсутствии файла, он будет автоматически создан. Поменяй способ взаимодействия и запусти код. В files google colab должен появиться новый файл:

In [ ]:
with open('new_file.json', 'r') as f:  # Поменяй r на w
  pass  # pass значит отсутствие действия

Теперь импортируем библиотеку json и воспользуемся методом dump. В files google colab должен появиться массив после выполнения следующего кода:

In [ ]:
import json

with open('words_massive.json', 'w') as f:
  json.dump(brown_freq_keys, f)  # В качестве аргумента функции указываем то,
                                 # что мы хотим сохранить, то есть список слов
                                 # и то, куда мы хотим сохранить, то есть в
                                 # созданный нам файл f

Вы можете открыть его двойным щелчком и посмотреть, что там действительно внутри все слова из нашего корпуса. Однако нам нужно будет использовать этот массив на python, а поэтому придётся загрузить json файл в переменную с помощью библиотеки. Для этого напиши простой код:

1. Открой файл words_massive для чтения.
3. Присвой переменной words_massive значение функции `json.load()`, указав в аргументе файл.

In [ ]:
# Запусти этот код, чтобы проверить сколько слов в нашем списке

print(len(words_massive))

### Создание функции
Только что ты создал в отдельных ячейках кода создание списка слов из корпуса. Но нам будет удобнее поместить это в функцию, которую мы сможем легко перемещать и использовать. Что такое функции и с чем их едят? Разберёмся на примере:

In [ ]:
def sum_numbers(a, b):
  return a + b

print(sum_numbers(5, 10))

Вот наглядный пример функции, которая складывает 2 числа, которые передаются ей в качестве аргумента, то есть те числа, которые мы пишем внутри круглых скобок функции:

```sum_numbers(5, 10)  # 5 и 10 это числа, они же аргумент функции```

Напиши свою функцию, которая будет отнимать числа:

Скорее всего у тебя получилось повторить пример. Давай разберём, как ты это сделал:

1. `def` это такое кодовое слово, с которого мы начинаем нашу функцию.
2. После `def` идёт название функции. Оно может быть каким угодно, но лучше не повторять названий уже существующих функций, иначе можно натворить дел.
3. После названия в круглых скобках указываем аргумент. Его может вовсе не быть, он может быть необязательным (значения указаны по умолчанию) или он может быть обязательным, как в нашем примере.
4. Заканчивается всё двоеточием, после которого у нас начинается "территория" нашей функции, в которую входят строки, которые начинаются с нескольких пробелов от начала и заканчивается теми строками, которые находятся на уровне def:
```
def blah_blah():
      # территория функции
      # территория функции
      # территория функции
# территория вне функции
```
5. В "теле" функции мы определяем действия, которые будет совершать функция при вызове. В нашем примере это return, то есть мы получим обратно какое-то значение, в нашем случае разность двух чисел.

Чтобы проверить, как ты усвоил информацию, создай функцию
`get_words_massive()`, которая будет открывать файл words_massive.json и читать массив слов в переменную. Переменную нужно будет вернуть с помощью return

In [ ]:
def get_words_massive():
  # Открой words_massive.json и с помощью json загрузи массив в переменную


  # Верни переменную с помощью return

In [ ]:
# Запусти этот код, чтобы проверить сколько слов в нашем массиве
print(len(get_words_massive()))

Поздравляю, ты отлично справился/справилась :)
Теперь, пожалуйста, скачай файл words_massive.json в корневую папку проекта, нажав по нему левой кнопкой мыши, а затем "скачать"/"download"